<a href="https://colab.research.google.com/github/kstyle2198/NLP_TIPS/blob/main/%EA%B2%BD%EC%9F%81%EC%82%ACNews_Sentiment%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 검토 프로세스
- Google Alert으로 경쟁사 뉴스를 수집한다.
- IFTTT를 활용하여 수집된 뉴스를 Google Sheet에 저장(모음)

- 이하 아래 코드 부분
- Google API 활용하여 Python 으로 Google Sheet 데이터 불러온다.(service_account.json 파일 생성)
- Bert 모델 불러온후, Fine tunning 한다.
- Best Model을 활용하여 경쟁사 뉴스에 대해 감정 분류를 한다.

## Google API 활용법 참고 (service_account.json 파일 생성법)

https://www.youtube.com/watch?v=bu5wXjz2KvU

# 라이브러리 설치

In [1]:
!pip install gspread==3.6.0

  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2


# Google Sheet 연동

In [2]:
import gspread

In [3]:
!pwd

/content


In [4]:
sa = gspread.service_account("/content/service_account.json")

In [5]:
sh = sa.open("한화오션")  #"삼성중공업" 도 가능
sh

<Spreadsheet '한화오션' id:1Cb5uFPUAehliwg5kwNg2-BFdy9NLT2wln586lO9PZx4>

In [6]:
wks = sh.worksheet("Sheet1")
wks

<Worksheet 'Sheet1' id:0>

In [7]:
print(f"Rows: {wks.row_count}")
print(f"Cols: {wks.col_count}")

Rows: 1196
Cols: 26


In [8]:
print(wks.acell("B9").value)

한화오션 뿌리산업특화단지 지원…협력사 상생 앞장 - 중소기업신문


In [9]:
print(wks.cell(9 ,2).value)

한화오션 뿌리산업특화단지 지원…협력사 상생 앞장 - 중소기업신문


In [10]:
print(wks.get('B2:B9'))

[['[단독] 카타르 LNG 프로젝트 2차 발주 본격화...한화오션·삼성重 수주 전망 - 더구루'], ['한화오션, 뿌리산업 협력사와 상생 - 시사포커스'], ['한화오션, 뿌리산업특화단지 지원…협력사 숙소 리모델링 - 아주경제'], ['한화오션, 거제 뿌리산업특화단지 지원사업으로 협력사 상생 앞장 - 서울신문'], ["국가기밀' 한화오션 잠수함 설계도 대만에… 경찰 수사 착수 - 서울와이어"], ['한화오션, 협력사와 상생 강화…직원 아파트 100세대 새단장 - 노컷뉴스'], ['"거제 근로자 정착 여건 개선"...한화오션, 뿌리산업특화단지 지원 앞장 - 뉴스퀘스트'], ['한화오션 뿌리산업특화단지 지원…협력사 상생 앞장 - 중소기업신문']]


In [11]:
wks.get_all_records()

[{'January 05, 2024 at 11:03AM': 'January 05, 2024 at 11:08AM',
  '한화오션, 뿌리산업특화단지 지원사업으로 협력사 상생 앞장 - 이투데이': '[단독] 카타르 LNG 프로젝트 2차 발주 본격화...한화오션·삼성重 수주 전망 - 더구루',
  'https://www.etoday.co.kr/news/view/2318383': 'https://www.theguru.co.kr/news/article.html?no=64776',
  '<b>한화오션</b> 거제사업장 전경. (사진제공=<b>한화오션</b>)<b>한화오션</b>은 뿌리산업특화단지 지원사업을 통해 협력사와의 상생에 나선다고 5일 밝혔다.뿌리산업...': '지난해 10월 HD현대중공업과 17척에 대한 건조계약을 체결하면서 삼성중공업과 <b>한화오션</b>과도 연내 계약을 예상했지만 해를 넘겨 계약이 성사될 전망이다. 5일&nbsp;...',
  'Google Alert - 한화오션': 'Google Alert - 한화오션',
  'https://www.google.com/alerts/feeds/03846553419935279766/11817066535797571903': 'https://www.google.com/alerts/feeds/03846553419935279766/11817066535797571903'},
 {'January 05, 2024 at 11:03AM': 'January 05, 2024 at 11:10AM',
  '한화오션, 뿌리산업특화단지 지원사업으로 협력사 상생 앞장 - 이투데이': '한화오션, 뿌리산업 협력사와 상생 - 시사포커스',
  'https://www.etoday.co.kr/news/view/2318383': 'https://www.sisafocus.co.kr/news/articleView.html?idxno=310672',
  '<b>한화오션</b> 거제사업장 전경. (사진제공=<b>한화오션</b>)<b>한화오션</b>은 뿌

In [12]:
list_data = wks.get_all_values()
list_data

[['January 05, 2024 at 11:03AM',
  '한화오션, 뿌리산업특화단지 지원사업으로 협력사 상생 앞장 - 이투데이',
  'https://www.etoday.co.kr/news/view/2318383',
  '<b>한화오션</b> 거제사업장 전경. (사진제공=<b>한화오션</b>)<b>한화오션</b>은 뿌리산업특화단지 지원사업을 통해 협력사와의 상생에 나선다고 5일 밝혔다.뿌리산업...',
  'Google Alert - 한화오션',
  'https://www.google.com/alerts/feeds/03846553419935279766/11817066535797571903'],
 ['January 05, 2024 at 11:08AM',
  '[단독] 카타르 LNG 프로젝트 2차 발주 본격화...한화오션·삼성重 수주 전망 - 더구루',
  'https://www.theguru.co.kr/news/article.html?no=64776',
  '지난해 10월 HD현대중공업과 17척에 대한 건조계약을 체결하면서 삼성중공업과 <b>한화오션</b>과도 연내 계약을 예상했지만 해를 넘겨 계약이 성사될 전망이다. 5일&nbsp;...',
  'Google Alert - 한화오션',
  'https://www.google.com/alerts/feeds/03846553419935279766/11817066535797571903'],
 ['January 05, 2024 at 11:10AM',
  '한화오션, 뿌리산업 협력사와 상생 - 시사포커스',
  'https://www.sisafocus.co.kr/news/articleView.html?idxno=310672',
  '',
  'Google Alert - 한화오션',
  'https://www.google.com/alerts/feeds/03846553419935279766/11817066535797571903'],
 ['January 05, 2024 at 11:14AM',
  '한화오션, 뿌리산업특화

# Google Sheet to DataFrame

In [13]:
import pandas as pd
columns = ['Timestamp', 'Title', 'URL', 'Content', 'Source', 'Source URL']
df = pd.DataFrame(list_data, columns=columns)
df.head()

,Timestamp,Title,URL,Content,Source,Source URL
0,"January 05, 2024 at 11:03AM","한화오션, 뿌리산업특화단지 지원사업으로 협력사 상생 앞장 - 이투데이",https://www.etoday.co.kr/news/view/2318383,<b>한화오션</b> 거제사업장 전경. (사진제공=<b>한화오션</b>)<b>한화오...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...
1,"January 05, 2024 at 11:08AM",[단독] 카타르 LNG 프로젝트 2차 발주 본격화...한화오션·삼성重 수주 전망 -...,https://www.theguru.co.kr/news/article.html?no...,지난해 10월 HD현대중공업과 17척에 대한 건조계약을 체결하면서 삼성중공업과 <b...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...
2,"January 05, 2024 at 11:10AM","한화오션, 뿌리산업 협력사와 상생 - 시사포커스",https://www.sisafocus.co.kr/news/articleView.h...,,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...
3,"January 05, 2024 at 11:14AM","한화오션, 뿌리산업특화단지 지원…협력사 숙소 리모델링 - 아주경제",https://www.ajunews.com/view/20240105102425710,<b>한화오션</b>은 협력사와의 상생 차원에서 거제사업장 뿌리산업특화단지 지원 사...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...
4,"January 05, 2024 at 11:15AM","한화오션, 거제 뿌리산업특화단지 지원사업으로 협력사 상생 앞장 - 서울신문",https://www.seoul.co.kr/news/newsView.php?id=2...,뿌리산업특화단지 사업 추진해 숙소 리모델링·커뮤니티센터 개소 노동자 정주여건 개선....,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...


In [14]:
def convert_datetime(date_string):
  datetime_value = pd.to_datetime(date_string, format='%B %d, %Y at %I:%M%p')
  return datetime_value

In [15]:
df["Timestamp"] = df["Timestamp"].apply(convert_datetime)

In [16]:
df['date'] = df["Timestamp"].dt.date
df['month'] = df["Timestamp"].dt.month

In [17]:
df.tail(2)

,Timestamp,Title,URL,Content,Source,Source URL,date,month
194,2024-01-09 07:30:00,“태영發 채권시장 불안 없었다”‥KCC·한화에어로에 조단위 '뭉칫돈' - 아시아경제,https://cm.asiae.co.kr/article/202401090717124...,두 회사 모두 채권 수요를 반영해 회사채를 증액 발행했다. 특히 한화에어로스페이스는...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-01-09,1
195,2024-01-09 07:40:00,수주 전략 바꾼 조선업계…'내실 다지기' 나선 이유 - 비즈워치,http://news.bizwatch.co.kr/article/industry/20...,삼성중공업과 <b>한화오션</b>도 수익성 위주 수주 전략을 펼치겠다는 구상이다. ...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-01-09,1


In [18]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
add_stopwords = ["", ".", "...", "·", "for", "to", "with", "is", "%", "(", ")", "?", "[", "]", "{", "}", ",", ";", ":", "-", "./", "/", "”","’", "*", "&", "“", "``", "'",
                "내", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", '==', 'etc…', 'p+',
                "kg㎠", "shall", "one", "two", "three"]

In [20]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

def clean_text(
    string:str,
    punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
    stop_words=['the', 'a', 'and', 'is', 'be', 'will']) -> str:
    """
    A method to clean text
    """
    global add_stopwords
    stop_words_list = stopwords.words('english')
    stop_words_list = stop_words_list + add_stopwords

    # Cleaning the urls
    string = re.sub(r'https?://\S+|www\.\S+', '', string)

    # Cleaning the html elements
    string = re.sub(r'<.*?>', '', string)

    # 한글 제거
    # string = re.sub(r'[가-힣]+', '', string)

    # 텍스트내 숫자만 제거
    # string1=""
    # for t in string:
    #     t1 = re.sub(r'\b\d+\b', '', t)
    #     string1 += t1
    # string = string1

    # Removing the punctuations
    for x in string.lower():
        # x = re.sub(r'\b\d+\b', '', x)
        if x in punctuations:
            string = string.replace(x, "")

    # Converting the text to lower
    string = string.lower()

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()


    return string

In [21]:
df["Content"] = df["Content"].apply(clean_text)

In [22]:
df.tail()

,Timestamp,Title,URL,Content,Source,Source URL,date,month
191,2024-01-09 06:12:00,"[꿈의 K방산]현실로 다가온 200억달러 수출, 세계 4강 / 머니투데이방송 (뉴스...",https://www.youtube.com/watch?v=P3W6a81Zp_Y,08 kbs news new 42k views middot 3805 middot g...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-01-09,1
192,2024-01-09 06:21:00,제2 전성기' 연 K-조선이 수주 목표 낮춘 이유 - 중공업 - 굿모닝경제,http://www.goodkyung.com/news/articleView.html...,hd한국조선해양·삼성중공업·한화오션 등 국내 조선3사는 지난해 3분기 11년 만에 ...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-01-09,1
193,2024-01-09 07:05:00,중국이 절대 따라올 수 없는 기술로 - 뉴스웨이,https://www.newsway.co.kr/news/view?ud=2024010...,hd한국조선해양은 2020년 이후 3년 만에 흑자를 달성했고 삼성중공업과 한화오션은...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-01-09,1
194,2024-01-09 07:30:00,“태영發 채권시장 불안 없었다”‥KCC·한화에어로에 조단위 '뭉칫돈' - 아시아경제,https://cm.asiae.co.kr/article/202401090717124...,두 회사 모두 채권 수요를 반영해 회사채를 증액 발행했다 특히 한화에어로스페이스는 ...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-01-09,1
195,2024-01-09 07:40:00,수주 전략 바꾼 조선업계…'내실 다지기' 나선 이유 - 비즈워치,http://news.bizwatch.co.kr/article/industry/20...,삼성중공업과 한화오션도 수익성 위주 수주 전략을 펼치겠다는 구상이다 목표 낮춘 조선...,Google Alert - 한화오션,https://www.google.com/alerts/feeds/0384655341...,2024-01-09,1


# Sentiment Classification

https://github.com/park-gb/financial-news-sentiment-classifier/blob/main/sentiment_classifier.ipynb

In [23]:
!pip install transformers
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.0 MB/s eta 0:00:00


In [24]:

import os
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import urllib.request
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [25]:
MODEL_NAME = "klue/bert-base"
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, from_pt=True)
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

In [26]:
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
  print("GPU 작동 중")
  mirrored_strategy = tf.distribute.MirroredStrategy()
else:
  print("GPU 미작동 중")

GPU 작동 중


In [27]:
# fine-tunning 데이터 준비
DATASET_URL = "https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv"
DATASET_NAME = "finance_data.csv"

In [28]:
urllib.request.urlretrieve(DATASET_URL,
                           filename = DATASET_NAME
                           )


('finance_data.csv', <http.client.HTTPMessage at 0x7eb47065d5a0>)

In [29]:
dataset = pd.read_csv(DATASET_NAME)
dataset.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [30]:
del dataset['sentence']

In [31]:
dataset['labels'] = dataset['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])
dataset.head()

,labels,kor_sentence
0,0,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [32]:
# 중복 데이터 확인
dataset[dataset['kor_sentence'].duplicated()]

,labels,kor_sentence
79,1,텔레콤월드와이어-2006년 4월 7일-TJ 그룹은 모닝 디지털 디자인 Oy 핀란드 ...
789,0,그룹의 사업은 스포츠의 광범위한 포트폴리오와 모든 주요 시장에서의 입지에 의해 균형...
1099,0,이 발표 내용에 대한 책임은 전적으로 발행자에게 있습니다.
1394,0,"핀란드 헬싱키에 본사를 둔 레민카이넨 그룹은 토목 공학, 건축 계약, 기술 건축 서..."
1416,0,"이 보고서는 블랙 앤 데커, 피스카스, 피스카스 브랜드, 후스크바르나 아웃도어 프로..."
2396,0,알스트롬의 주가는 나스닥 OMX 헬싱키에서 인용되고 있다.
2567,0,SSH 통신 보안 코퍼레이션은 핀란드 헬싱키에 본사를 두고 있다.
2889,0,재정적인 세부사항은 공개되지 않았다.
2890,0,재정적인 세부사항은 공개되지 않았다.
2892,0,금융 조건은 공개되지 않았다.


In [33]:

# DATASET_PREP_FILE = './data/dataset_prep.csv'

# 중복 데이터 제거
# dataset.drop_duplicates(subset = ['kor_sentence'], inplace = True)
# dataset.to_csv(DATASET_PREP_FILE) # 구글 드라이브 내 data 폴더에 저장

In [34]:
# 입출력 데이터 분리
X_data = dataset['kor_sentence']
y_data = dataset['labels']

In [35]:
TEST_SIZE = 0.2 # Train: Test = 8 :2 분리
RANDOM_STATE = 42
# strtify = True 일 경우, 데이터 분리 이전의 라벨별 분포 고려
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data,
                                                    test_size = TEST_SIZE,
                                                    random_state = RANDOM_STATE,
                                                    stratify = y_data)


In [36]:
print(f"훈련 입력 데이터 개수: {len(X_train)}")
print(f"테스트 입력 데이터 개수: {len(X_test)}")

훈련 입력 데이터 개수: 3876
테스트 입력 데이터 개수: 970


In [37]:
X_test

2898    2008년 1월 3일 핀란드 전자제품 제조회사인 인틱 오이(Inctive Oyj)는...
1271    Jon Risfelt는 49세로 스웨덴 왕립 공과대학교에서 화학공학 석사 학위를 받...
2467       글래스턴의 주식 GLA1V는 나스닥 OMX 헬싱키 미드 캡 리스트에 상장되어 있다.
3154    라헤에 계획 중인 5,000 메가와트 풍력 발전소는 루키 라헤 작업소 앞바다에 건설...
521                            이 명령에는 시동 및 시운전 서비스도 포함된다.
                              ...                        
3603    이날 오후 1시 33분 뉴욕증시에서 가민은 4.5% 하락한 34.53달러를 기록했으...
115     노키아도 휴대전화 수익률이 1년 전 분기 13%에서 22% 이상으로 상승했음에도 불...
2577    테크노트리의 컨버전스 충전 솔루션에는 선불 및 후불 청구, 음성 통화, 영상 통화,...
3576      엘코텍은 2010년 4/4분기 순매출액이 3/4분기 수준이 될 것으로 예상하고 있다.
2754            핀란드 루메네 화장품 기업 오너인 사모펀드 캡맨이 루메네 상장을 추진한다.
Name: kor_sentence, Length: 970, dtype: object

In [38]:

# 입력 데이터(문장) 길이 제한
MAX_SEQ_LEN = 64

In [39]:

def convert_data(X_data, y_data):
    # BERT 입력으로 들어가는 token, mask, segment, target 저장용 리스트
    tokens, masks, segments, targets = [], [], [], []

    for X, y in tqdm(zip(X_data, y_data)):
        # token: 입력 문장 토큰화
        token = tokenizer.encode(X, truncation = True, padding = 'max_length', max_length = MAX_SEQ_LEN)

        # Mask: 토큰화한 문장 내 패딩이 아닌 경우 1, 패딩인 경우 0으로 초기화
        num_zeros = token.count(0)
        mask = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros

        # segment: 문장 전후관계 구분: 오직 한 문장이므로 모두 0으로 초기화
        segment = [0]*MAX_SEQ_LEN

        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        targets.append(y)

    # numpy array로 저장
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

In [40]:
# train 데이터를 Bert의 Input 타입에 맞게 변환
train_x, train_y = convert_data(X_train, y_train)

3876it [00:02, 1391.93it/s]


In [41]:
# test 데이터를 Bert의 Input 타입에 맞게 변환
test_x, test_y = convert_data(X_test, y_test)


970it [00:00, 1462.35it/s]


In [42]:
# token, mask, segment 입력 정의
token_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_word_ids')
mask_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_masks')
segment_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_segment')
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])


In [43]:
bert_outputs

TFSequenceClassifierOutput(loss=None, logits=<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'tf_bert_for_sequence_classification')>, hidden_states=None, attentions=None)

In [44]:
bert_output = bert_outputs[0]
bert_output

<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'tf_bert_for_sequence_classification')>

In [45]:
DROPOUT_RATE = 0.5
NUM_CLASS = 3
dropout = tf.keras.layers.Dropout(DROPOUT_RATE)(bert_output)
# Multi-class classification 문제이므로 activation function은 softmax로 설정
sentiment_layer = tf.keras.layers.Dense(NUM_CLASS, activation='softmax', kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev=0.02))(dropout)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_layer)

In [46]:
# 옵티마이저 Rectified Adam 하이퍼파리미터 조정
OPTIMIZER_NAME = 'RAdam'
LEARNING_RATE = 5e-5
TOTAL_STEPS = 10000
MIN_LR = 1e-5
WARMUP_PROPORTION = 0.1
EPSILON = 1e-8
CLIPNORM = 1.0
optimizer = tfa.optimizers.RectifiedAdam(learning_rate = LEARNING_RATE,
                                          total_steps = TOTAL_STEPS,
                                          warmup_proportion = WARMUP_PROPORTION,
                                          min_lr = MIN_LR,
                                          epsilon = EPSILON,
                                          clipnorm = CLIPNORM)

In [47]:
# 감정분류 모델 컴파일
sentiment_model.compile(optimizer = optimizer,
                        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                        metrics = ['accuracy'])

In [48]:
MIN_DELTA = 1e-3
PATIENCE = 5

early_stopping = EarlyStopping(
    monitor = "val_accuracy",
    min_delta = MIN_DELTA,
    patience = PATIENCE)

In [49]:
# 최고 성능의 모델 파일을 저장할 이름과 경로 설정
BEST_MODEL_NAME = './model/best_model.h5'

In [50]:
model_checkpoint = ModelCheckpoint(
    filepath = BEST_MODEL_NAME,
    monitor = "val_loss",
    mode = "min",
    save_best_only = True, # 성능 향상 시에만 모델 저장
    verbose = 1
)

In [51]:
callbacks = [early_stopping, model_checkpoint]

In [52]:
EPOCHS = 100
BATCH_SZIE = 32

In [53]:
sentiment_model.fit(train_x, train_y,
                    epochs = EPOCHS,
                    shuffle = True,
                    batch_size = BATCH_SZIE,
                    validation_data = (test_x, test_y),
                    callbacks = callbacks
                    )

Epoch 1/100
122/122 [==============================] - ETA: 0s - loss: 1.0941 - accuracy: 0.4360
Epoch 1: val_loss improved from inf to 1.08825, saving model to ./model/best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


122/122 [==============================] - 79s 346ms/step - loss: 1.0941 - accuracy: 0.4360 - val_loss: 1.0883 - val_accuracy: 0.4155
Epoch 2/100
122/122 [==============================] - ETA: 0s - loss: 1.0780 - accuracy: 0.4809
Epoch 2: val_loss improved from 1.08825 to 1.06250, saving model to ./model/best_model.h5
122/122 [==============================] - 32s 259ms/step - loss: 1.0780 - accuracy: 0.4809 - val_loss: 1.0625 - val_accuracy: 0.6515
Epoch 3/100
122/122 [==============================] - ETA: 0s - loss: 1.0470 - accuracy: 0.5531
Epoch 3: val_loss improved from 1.06250 to 1.00868, saving model to ./model/best_model.h5
122/122 [==============================] - 32s 260ms/step - loss: 1.0470 - accuracy: 0.5531 - val_loss: 1.0087 - val_accuracy: 0.7454
Epoch 4/100
122/122 [==============================] - ETA: 0s - loss: 0.9907 - accuracy: 0.6133
Epoch 4: val_loss improved from 1.00868 to 0.94093, saving model to ./model/best_model.h5
122/122 [============================

In [54]:
# 최고 성능의 모델 불러오기
sentiment_model_best = tf.keras.models.load_model(BEST_MODEL_NAME,
                                                  custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})

In [55]:
len(test_x), len(test_x[0]), len(test_x[1]), len(test_x[2])

(3, 970, 970, 970)

In [56]:
# 모델이 예측한 라벨 도출
predicted_value = sentiment_model_best.predict(test_x)
predicted_label = np.argmax(predicted_value, axis = 1)

31/31 [==============================] - 6s 60ms/step


In [57]:
predicted_value

array([[0.82658476, 0.09923814, 0.07417702],
       [0.85342646, 0.07803962, 0.0685339 ],
       [0.85527515, 0.0770254 , 0.06769949],
       ...,
       [0.8556316 , 0.07689814, 0.06747022],
       [0.8540644 , 0.07754609, 0.06838942],
       [0.85310423, 0.07981224, 0.06708357]], dtype=float32)

In [58]:

# Classification Report 저장
# CL_REPORT_FILE = "./metric/cl_report.csv"

cl_report = classification_report(test_y, predicted_label, output_dict = True)
cl_report_df = pd.DataFrame(cl_report).transpose()
cl_report_df = cl_report_df.round(3)
# cl_report_df.to_csv(CL_REPORT_FILE)
print(cl_report_df)


              precision  recall  f1-score  support
0                 0.875   0.859     0.867  576.000
1                 0.751   0.762     0.756  273.000
2                 0.780   0.818     0.798  121.000
accuracy          0.827   0.827     0.827    0.827
macro avg         0.802   0.813     0.807  970.000
weighted avg      0.828   0.827     0.827  970.000


In [59]:
df["text"] = df["Title"]+" "+df["Content"]
df["text"]

0      한화오션, 뿌리산업특화단지 지원사업으로 협력사 상생 앞장 - 이투데이 한화오션 거제...
1      [단독] 카타르 LNG 프로젝트 2차 발주 본격화...한화오션·삼성重 수주 전망 -...
2                            한화오션, 뿌리산업 협력사와 상생 - 시사포커스 
3      한화오션, 뿌리산업특화단지 지원…협력사 숙소 리모델링 - 아주경제 한화오션은 협력사...
4      한화오션, 거제 뿌리산업특화단지 지원사업으로 협력사 상생 앞장 - 서울신문 뿌리산업...
                             ...                        
191    [꿈의 K방산]현실로 다가온 200억달러 수출, 세계 4강 / 머니투데이방송 (뉴스...
192    제2 전성기' 연 K-조선이 수주 목표 낮춘 이유 - 중공업 - 굿모닝경제 hd한국...
193    중국이 절대 따라올 수 없는 기술로 - 뉴스웨이 hd한국조선해양은 2020년 이후 ...
194    “태영發 채권시장 불안 없었다”‥KCC·한화에어로에 조단위 '뭉칫돈' - 아시아경제...
195    수주 전략 바꾼 조선업계…'내실 다지기' 나선 이유 - 비즈워치 삼성중공업과 한화오...
Name: text, Length: 196, dtype: object

In [60]:
def convert_data1(X):
    # BERT 입력으로 들어가는 token, mask, segment, target 저장용 리스트
    tokens, masks, segments = [], [], []

    # token: 입력 문장 토큰화
    token = tokenizer.encode(X, truncation = True, padding = 'max_length', max_length = MAX_SEQ_LEN)

    # Mask: 토큰화한 문장 내 패딩이 아닌 경우 1, 패딩인 경우 0으로 초기화
    num_zeros = token.count(0)
    mask = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros

    # segment: 문장 전후관계 구분: 오직 한 문장이므로 모두 0으로 초기화
    segment = [0]*MAX_SEQ_LEN


    tokens.append(token)
    masks.append(mask)
    segments.append(segment)

    # numpy array로 저장
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)

    # numpy array로 저장
    tokens = np.array(token)
    masks = np.array(mask)
    segments = np.array(segment)

    return [tokens, masks, segments]

In [72]:
target_news = df["text"].sample(1).values[0]
print(target_news)
target1 = convert_data1(target_news)
print(target1)
target2 = [target1[0].reshape(-1,64), target1[1].reshape(-1,64), target1[2].reshape(-1,64)]
target2
predicted_value = sentiment_model_best.predict(target2)
predicted_value

“안 팔리면 쪼개 판다”…'분리매각' 늘어나는 이유 - 이데일리 한화오션·전주페이퍼 통매각 성공하기도 이데일리 허지은 기자 최근 결국 대우조선해양은 분리매각없이 한화그룹에 통매각돼 한화오션으로 신규 출범했다
[array([    2,   116,  1378, 10426,  2460, 13321, 18230,   117,   121,
          11,  5590,  2077,  2215,    11,  5351,  2259,  3782,    17,
        8568,  6514,  5621,  2168,  2197,   100,  6868, 13907,  2712,
        1799,  2077,  2215,  3920, 31302,  2119,  8568,  6514,  1905,
        2118,  2073,  4055,  3744,  3983,  5438, 11836,  2097,  2221,
        2073,  5590,  2077,  2215,  4087, 19442,  2170,  1799,  2077,
        2215,  2811,  5621,  2168,  2197,  6233,  5089,  5788,  2371,
           3]), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 

array([[0.36536175, 0.49231485, 0.14232334]], dtype=float32)

In [62]:
['neutral', 'positive', 'negative']

['neutral', 'positive', 'negative']